In [0]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch7
!touch ch7/__init__.py

mkdir: cannot create directory ‘ch7’: File exists


- 函数装饰器用于在源码中"标记"函数， 以某种方式增强函数的行为，其实现基础是闭包
- 闭包是回调式异步编程和函数式编程风格的基础

## 7.1 装饰器基础知识

- 装饰器是可调用对象，其参数是另一个被装饰的函数
  - 装饰器可能会处理被装饰的函数，然后把它返回
  - 或者将其替换成另一个函数或可调用对象
- 装饰器是语法糖，其可以像常规的可调用对象那样调用，参数是另一个函数
  - 在元编程（运行时改变程序的行为） 时应用较多

###### 示例 7-1 装饰器通常把函数替换为另外一个函数

In [0]:
def deco(func):
  def inner():
    print('running inner()')
  return inner  # deco 返回 inner 函数对象


@deco
def target():
  print('running target()')


def target1():
  print('running target1()')

target1 = deco(target1)  # 与 @deco 的写法等效

In [0]:
target()

running inner()


In [0]:
target1()

running inner()


In [0]:
repr(target)  # target 现在是 inner 对象的引用

'<function deco.<locals>.inner at 0x7f331d553bf8>'

In [0]:
repr(target1)  # target1 现在也是 inner 对象的引用

'<function deco.<locals>.inner at 0x7f331ea27620>'

## 7.2 Python 何时执行装饰器

- 装饰器在被装饰的函数定义之后立即执行
  - 通常在导入时执行（即 Python 加载模块时）
- 而被装饰的函数只在明确调用时运行

###### 示例 7-2 registration.py

In [0]:
%%writefile ch7/registration.py
registry = []  # 用来保存被 @register 装饰的函数引用

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func


@register
def f1():
  print('running f1()')


@register
def f2():
  print('running f2()')


def f3():
  print('running f3()')


def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()


if __name__ == "__main__":
  main()

Overwriting ch7/registration.py


- 以脚本运行

In [0]:
!python ch7/registration.py

running register(<function f1 at 0x7f3f81105d08>)
running register(<function f2 at 0x7f3f81105d90>)
running main()
registry -> [<function f1 at 0x7f3f81105d08>, <function f2 at 0x7f3f81105d90>]
running f1()
running f2()
running f3()


- 导入模块

In [0]:
import ch7.registration

running register(<function f1 at 0x7f331d548ea0>)
running register(<function f2 at 0x7f331d57f6a8>)


## 7-3 使用装饰器改进‘策略’模式

- 与 6.1 节给出的方案相比，使用装饰器有以下几个优点
  - 促销策略函数无需使用特殊的名称（即不用以 _promo 结尾）
  - `@promotion` 装饰器突出了被装饰的函数的作用
    - 便于临时禁用某个促销策略：只需把装饰器注释掉
  - 促销折扣策略可以在其他模块中定义，只要使用 `@promotion` 装饰即可。

###### 示例 7-3 `promos` 列表中的值使用 `promotion` 装饰器进行填充

In [0]:
%%writefile ch7/promos.py
promos = []


def promotions(promo_func):
  promos.append(promo_func)
  return promo_func


@promotions
def fidelity_promo(order):  # 第一个具体策略
  """为积分为 1000 或以上的顾客提供 5% 的折扣"""
  return order.total() * 0.05 if order.customer.fidelity > 1000 else 0


@promotions
def bulk_item_promo(order):  # 第二个策略
  """单个商品为 20 个或以上时，提供 10% 折扣"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount


@promotions
def large_order_promo(order):  # 第三个策略
  """订单中的不同商品达到10个或以上时提供7%折扣"""
  distinct_items = {item.product for item in order.cart}
  return order.total() * 0.07 if len(distinct_items) >= 10 else 0


def best_promo(order):
  """选择可用的最佳策略
  """
  return max(promo(order) for promo in promos)

Writing ch7/promos.py


## 7-4 变量作用域规则

- Python 不要求声明变量，但是假定在函数定义体中赋值的变量是局部变量
- 可以使用 `global` 声明全局变量

###### 示例 7-4 一个函数，读取一个局部变量和一个全局变量

In [0]:
def f1(a):
  print(a)
  print(b)

In [0]:
f1(3)

3


NameError: ignored

- 如果先给全局变量 $b$ 赋值，则函数能正常运行

In [0]:
b = 6
f1(3)

3
6


###### 示例 7-5 b 是局部变量，因为在函数的定义体中给它赋值了

In [0]:
b = 6
def f2(a):
  print(a)
  print(b)
  b = 9

In [0]:
f2(3)

3


UnboundLocalError: ignored

- Python 编译函数的定义体时，它判断 `b` 是局部变量，因为在函数中给它赋值了

- 使用 `global` 声明把 $b$ 当成全局变量

In [0]:
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

In [0]:
f3(3)

3
6


In [0]:
b

9

In [0]:
f3(3)

3
9


In [0]:
b = 30
b

30

###### 示例 7-6　反汇编示例 7-4 中的 f1 函数

In [0]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


- `LOAD_FAST` 为局部变量
- `LOAD_GLOBAL` 为全局变量

###### 示例 7-7　反汇编示例 7-5 中的 f2 函数

In [0]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## 7-5 闭包

- 闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量

###### 示例 7-8　average_oo.py：计算移动平均值的类

In [0]:
%%writefile ch7/average__oo.py
class Averager():
  """示例 7-8　average_oo.py：计算移动平均值的类
  """
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    return sum(self.series) / len(self.series)

Overwriting ch7/average__oo.py


In [0]:
from ch7.average__oo import Averager

In [0]:
avg = Averager()
avg(10)

10.0

In [0]:
avg(11)

10.5

In [0]:
avg(12)

11.0

###### 示例 7-9　average.py：计算移动平均值的高阶函数

In [0]:
%%writefile ch7/average.py
def make_averager():
  """示例 7-9　average.py：计算移动平均值的高阶函数"""
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series) 
    return total / len(series)

  return averager

Overwriting ch7/average.py


In [0]:
from ch7.average import make_averager

In [0]:
avg = make_averager()
avg(10)

10.0

In [0]:
avg(11)

10.5

In [0]:
avg(12)

11.0

##### 分析示例 7-9 中的函数

- `series` 是 `make_averager` 函数的局部变量
  - 因为在此函数的定义体中初始化了 `series`：`series = []`
  - 但调用 `avg(10)`时，`make_averager` 函数已经返回了，因此它的本地作用域不复存在
- 在 `averager` 函数中，`series` 是自由变量（free variable）
  - 自由变量指未在本地作用域中绑定的函数
  - 示意图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200402180929.png width=600>

- Python 在 `__code__` 属性（表示编译后的函数定义体）中保存局部变量和自由变量的名称
- 闭包是一种函数，它会保留定义函数时存在的自由变量的绑定
  - 如此调用函数时，虽然定义的作用域不可用了，但是仍能使用那些绑定

###### 示例 7-11　审查 make_averager（见示例 7-9）创建的函数

In [0]:
avg.__code__.co_varnames

('new_value', 'total')

In [0]:
avg.__code__.co_freevars

('series',)

- `avg.__closure__` 中的各个元素对应于 `avg.__code__.co_freevars` 中的一个名称。这些元素是 cell 对象，在其 `cell_contents` 属性中保存着真正的值

In [0]:
avg.__closure__

(<cell at 0x7f781b3d87c8: list object at 0x7f781b451488>,)

In [0]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## 7.6 `nonlocal` 声明

- `nonlocal` 可以将变量标记为自由变量

###### 示例 7-13　计算移动平均值的高阶函数，不保存所有历史值，但有缺陷

In [0]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count

  return averager

- 当 `count` 是不可变类型时， `count += 1` 相当于 `count = count + 1`， 此时会把 `count` 变为局部变量，从而导致程序运行报错
- 示例 7-9 没有报错是因为没有给 `series` 赋值，只是使用了 `series.append()`，本质上是利用了列表是可变对象这一事实

In [0]:
avg = make_averager()
avg(10)

UnboundLocalError: ignored

###### 示例 7-14　计算移动平均值，不保存所有历史（使用 nonlocal 修正）

In [0]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count

  return averager

In [0]:
avg = make_averager()

In [0]:
avg(10)

10.0

In [0]:
avg(11)

10.5

In [0]:
avg(12)

11.0

## 7.7 实现一个简单的装饰器

###### 示例 7-15　一个简单的装饰器，输出函数的运行时间

In [0]:
%%writefile ch7/clockdeco1.py
import time


def clock(func):
  """示例 7-15　一个简单的装饰器，输出函数的运行时间"""
  def clocked(*args):  # 支持任意个定位参数
    t0 = time.perf_counter()
    result = func(*args)  # func 相当于自由变量
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ", ".join(repr(arg) for arg in args)
    print('[%0.8fs]%s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked  # 返回内部函数，取低被装饰的函数

Writing ch7/clockdeco1.py


###### 示例 7-16　使用 clock 装饰器

In [0]:
%%writefile ch7/clockdeco_demo.py
"""示例 7-16　使用 clock 装饰器"""

import time
from ch7.clockdeco1 import clock


@clock
def snooze(seconds):
  time.sleep(seconds)


@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)


if __name__ == "__main__":
  print("*" * 40, 'Calling snozze(.123)')
  snooze(.123)
  print("*" * 40, 'Calling factorial(6)')
  print('6! = ', factorial(6))


Overwriting ch7/clockdeco_demo.py


In [0]:
!python -m ch7.clockdeco_demo

**************************************** Calling snozze(.123)
[0.12319565s]snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000092s]factorial(1) -> 1
[0.00001705s]factorial(2) -> 2
[0.00002761s]factorial(3) -> 6
[0.00003692s]factorial(4) -> 24
[0.00004579s]factorial(5) -> 120
[0.00005579s]factorial(6) -> 720
6! =  720


### 工作原理

In [0]:
from ch7.clockdeco_demo import factorial
factorial.__name__

'clocked'

- `factorial` 保存的是 `clocked` 的引用
- 这是装饰器典型的行为：把被装饰的函数替换为新函数，二者接受相同的参数，而且返回被装饰的函数本该返回的值，同时会作些额外操作
- 示例 7-15 中实现的 clock 装饰器有几个缺点
  1. 不支持关键字参数
  2. 遮盖了被装饰函数的 __name__ 和 __doc__ 属性
    - 可以使用 `functools.wraps` 装饰器把相关属性从 `func` 复制到 `clocked` 中

###### 示例 7-17　改进后的 clock 装饰器

In [0]:
%%writefile ch7/clockdeco2.py
"""示例 7-17　改进后的 clock 装饰器"""

import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    name = func.__name__
    arg_lst = []
    if args:
      arg_lst.append(", ".join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs]%s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

Writing ch7/clockdeco2.py


In [0]:
from ch7.clockdeco2 import clock

In [0]:
@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

In [0]:
factorial(6)

[0.00000143s]factorial(1) -> 1
[0.00125670s]factorial(2) -> 2
[0.00171876s]factorial(3) -> 6
[0.00263858s]factorial(4) -> 24
[0.00290537s]factorial(5) -> 120
[0.00314426s]factorial(6) -> 720


720

In [0]:
factorial.__name__

'factorial'

## 7.8 标准库中的装饰器

### 7.8.1 使用 `functools.lru_cache` 做备忘

- `functools.lru_cache` 实现了备忘功能，其将耗时的函数结果保存起来，避免传入相同参数时重复运算
  - `lru` 是 `Least Recently Used` 的缩写
- `lru_cache` 可以使用两个可选参数来配置
  - `functools.lru_cache(maxsize=128, typed=False)`
  - `maxsize` 参数指定存储多少个调用的结果。缓存满了之后，旧的结果会被扔掉，腾出空间
    - 为了得到最佳性能，`maxsize` 应该设为 2 的幂
  - `typed` 参数如果设为 `True`，把不同参数类型得到的结果分开保存
    - 即把通常认为相等的浮点数和整数参数（如 1 和 1.0）区分开
- 因为 lru_cache 使用字典存储结果，而且键根据调用时传入的定位参数和关键字参数创建，所以被 `lru_cache` 装饰的函数，它的所有参数都必须是可散列的。

###### 示例 7-18　生成第 `n` 个斐波纳契数，递归方式非常耗时

In [0]:
%%writefile ch7/fibonacci.py
"""示例 7-18　生成第 n 个斐波纳契数，递归方式非常耗时"""

from ch7.clockdeco2 import clock


@clock
def fibonacci(n):
  return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)


if __name__ == "__main__":
  print(fibonacci(6))

Writing ch7/fibonacci.py


In [0]:
!python -m ch7.fibonacci

[0.00000048s]fibonacci(0) -> 0
[0.00000143s]fibonacci(1) -> 1
[0.00023746s]fibonacci(2) -> 1
[0.00000024s]fibonacci(1) -> 1
[0.00000048s]fibonacci(0) -> 0
[0.00000048s]fibonacci(1) -> 1
[0.00001884s]fibonacci(2) -> 1
[0.00003672s]fibonacci(3) -> 2
[0.00030422s]fibonacci(4) -> 3
[0.00000024s]fibonacci(1) -> 1
[0.00000024s]fibonacci(0) -> 0
[0.00000024s]fibonacci(1) -> 1
[0.00001574s]fibonacci(2) -> 1
[0.00003123s]fibonacci(3) -> 2
[0.00000024s]fibonacci(0) -> 0
[0.00000024s]fibonacci(1) -> 1
[0.00001550s]fibonacci(2) -> 1
[0.00000024s]fibonacci(1) -> 1
[0.00000072s]fibonacci(0) -> 0
[0.00000048s]fibonacci(1) -> 1
[0.00003529s]fibonacci(2) -> 1
[0.00005078s]fibonacci(3) -> 2
[0.00008249s]fibonacci(4) -> 3
[0.00012851s]fibonacci(5) -> 5
[0.00044990s]fibonacci(6) -> 8
8


###### 示例 7-19　使用缓存实现，速度更快

In [0]:
%%writefile ch7/fibonacci_lru.py
"""示例 7-19　使用缓存实现，速度更快"""

import functools
from ch7.clockdeco2 import clock


@functools.lru_cache()  # 加括号的原因是 lur_cache 可以接受配置参数
@clock
def fibonacci(n):
  return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)


if __name__ == "__main__":
  print(fibonacci(6))


Overwriting ch7/fibonacci_lru.py


In [0]:
!python -m ch7.fibonacci_lru

[0.00000072s]fibonacci(0) -> 0
[0.00000119s]fibonacci(1) -> 1
[0.00009608s]fibonacci(2) -> 1
[0.00000119s]fibonacci(3) -> 2
[0.00012565s]fibonacci(4) -> 3
[0.00000072s]fibonacci(5) -> 5
[0.00015402s]fibonacci(6) -> 8
8


### 7.8.2 单分派泛函数

- `functools.singledispatch` 装饰器可以把整体方案拆分成多个模块，甚至可以为无法修改的类提供专门的函数
- 使用`@singledispatch` 装饰的普通函数会变成泛函数（generic function）
  - 根据第一个参数的类型，以不同方式执行相同操作的一组函数
- `singledispatch` 机制的一个显著特征是，可以在系统的任何地方和任何模块中注册专门函数
  - 如果后来在新的模块中定义了新的类型，可以轻松地添加一个新的专门函数来处理那个类型
  - 此外，还可以为不是自己编写的或者不能修改的类添加自定义函数

###### 示例 7-21　`singledispatch` 创建一个自定义的 `htmlize.register` 装饰器，把多个函数绑在一起组成一个泛函数

In [0]:
%%writefile ch7/htmlize.py
"""示例 7-21　singledispatch 创建一个自定义的 htmlize.register 装饰器，把多个函数绑在一起组成一个泛函数"""

from functools import singledispatch
from collections import abc
import numbers
import html


@singledispatch
def htmlize(obj):
  content = html.escape(repr(obj))
  return '<pre>{}</pre>'.format(content)


@htmlize.register(str)
def _(text):
  content = html.escape(text).replace('\n', '<br>\n')
  return '<p>{0}</p>'.format(content)


@htmlize.register(numbers.Integral)  # numbers.Inergal 是 int 的虛拟超类
def _(n):
  return '<pre>{0} (0x{0:x})</pre>'.format(n)


@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)  # 叠放多个 register 装饰器， 让同一个函数支持不同类型
def _(seq):
  inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
  return '<ul>\n<li>' + inner + '</li>\n</ul>'

Overwriting ch7/htmlize.py


- 只要可能，注册的专门函数应该处理抽象基类（如 `numbers.Integral`和 `abc.MutableSequence`），不要处理具体实现（如 `int` 和`list`）
 - 这样，代码支持的兼容类型更广泛

In [0]:
from ch7.htmlize import htmlize

In [0]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [0]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [0]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br>\n- a game</p>'

In [0]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [0]:
print(htmlize(['alpha', 66, {3, 2, 1}]))  # 根据各元素的类型进行格式化

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## 7-9 叠放装饰器

```python
@d1
@d2
def f():
  print('f')
```

等同于：
```python
def f():
  print('f')

f = d1(d2(f))
```

## 7-10 参数化装饰器

- 为了让装饰器接受其它参数，可以创建一个装饰器工厂函数，把参数传给它，返回一个装饰器，然后再把它应用到要装饰的函数上

 ### 7.10.1 一个参数化的注册装饰器 

- 示例 7-23 中新的 register 函数不是装饰器，而是装饰器工厂函数
  - 调用它会返回真正的装饰器，这才是应用到目标函数上的装饰器

###### 示例 7-23　为了接受参数，新的 register 装饰器必须作为函数调用

In [0]:
%%writefile ch7/registration_param.py
"""示例 7-23　为了接受参数，新的 register 装饰器必须作为函数调用"""

registry = set()


def register(active=True):
  def decorate(func):  # decorate 才是真正的装饰器，其接受一个函数作为参数
    print("running register(active=%s) -> decorate(%s)" %(active, func))
    if active:
      registry.add(func)
    else:
      registry.discard(func)  # 如果 active 为 false，且 func 在 registry 存在，则将其删除
    return func  # decorate 是装饰器，返回函数
  return decorate  # register 是装饰器工厂函数，返回装饰器


@register(active=False)  # @register 工厂函数必须作为函数调用，并且传入所需的参数
def f1():
  print('running f1()')


@register()
def f2():
  print('running f2()')


def f3():
  print('running f3()')


Writing ch7/registration_param.py


- `register()` 要返回 `decorate`，然后把它应用到被装饰的函数上。

In [0]:
import ch7.registration_param

running register(active=False) -> decorate(<function f1 at 0x7ff7988150d0>)
running register(active=True) -> decorate(<function f2 at 0x7ff798815950>)


###### 示例 7-24　使用示例 7-23 中的 registration_param 模块

In [0]:
from ch7.registration_param import *
registry

{<function ch7.registration_param.f2>}

In [0]:
register()(f3)

running register(active=True) -> decorate(<function f3 at 0x7ff7988152f0>)


<function ch7.registration_param.f3>

In [0]:
registry

{<function ch7.registration_param.f2>, <function ch7.registration_param.f3>}

In [0]:
register(active=False)(f2)

running register(active=False) -> decorate(<function f2 at 0x7ff798815950>)


<function ch7.registration_param.f2>

In [0]:
registry

{<function ch7.registration_param.f3>}

### 7.10.2 参数化 `clock` 装饰器

###### 示例 7-25　clockdeco_param.py 模块：参数化 `clock` 装饰器

In [0]:
%%writefile ch7/clockdeco_param.py
"""示例 7-25　clockdeco_param.py 模块：参数化 clock 装饰器"""

import time
import functools

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'


def clock(fmt=DEFAULT_FMT):
  def decorate(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
      t0 = time.time()
      _result = func(*args, **kwargs)
      elapsed = time.time() - t0
      name = func.__name__
      arg_lst = []
      if args:
        arg_lst.append(", ".join(repr(arg) for arg in args))
      if kwargs:
        pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
        arg_lst.append(', '.join(pairs))
      args = ', '.join(arg_lst)
      result = repr(_result)  # 便于显示
      print(fmt.format(**locals()))  # 在 fmt 中引用 clocked 的局部变量
      return _result
    return clocked
  return decorate


if __name__ == "__main__":
  @clock()
  def snozze(seconds):
    time.sleep(seconds)

  for i in range(3):
    snozze(.123)

Overwriting ch7/clockdeco_param.py


In [0]:
!python ch7/clockdeco_param.py

[0.12327051s] snozze(0.123) -> None
[0.12325883s] snozze(0.123) -> None
[0.12317467s] snozze(0.123) -> None


###### 示例 7-26　clockdeco_param_demo1.py

In [0]:
%%writefile ch7/clockdeco_demo1.py
"""示例 7-26　clockdeco_param_demo1.py"""

import time
from ch7.clockdeco_param import clock


@clock('{name}: {elapsed}s')
def snooze(seconds):
  time.sleep(seconds)


for i in range(3):
  snooze(.123)

Overwriting ch7/clockdeco_demo1.py


In [0]:
!python -m ch7.clockdeco_demo1

snooze: 0.12326526641845703s
snooze: 0.12318658828735352s
snooze: 0.12317943572998047s


###### 示例 7-26　clockdeco_param_demo1.py

In [0]:
%%writefile ch7/clockdeco_demo2.py
"""示例 7-26　clockdeco_param_demo1.py"""

import time
from ch7.clockdeco_param import clock


@clock('{name}({args}) dt={elapsed}s')
def snooze(seconds):
  time.sleep(seconds)


for i in range(3):
  snooze(.123)

Writing ch7/clockdeco_demo2.py


In [0]:
!python -m ch7.clockdeco_demo2

snooze(0.123) dt=0.12315654754638672s
snooze(0.123) dt=0.12327861785888672s
snooze(0.123) dt=0.12329649925231934s
